In [1]:
import numpy as np
import cv2
from pathlib import Path
from yolov11_onnx_wrapper import YOLOv11

In [2]:
model_path = Path("../yolo11s.onnx")
model = YOLOv11(
    model_path=model_path,
    valid_class_checker=lambda lbl_id, _: 1 <= lbl_id <= 8 # only detect vehicles
)
# vid_path_l = [ Path("../input_videos/5473765-uhd_3840_2160_24fps.mp4") ]
vid_path_l = Path("../input_videos").glob("*.mp4")
out_vid_dir = Path("./out_vid_dir")
out_vid_dir.mkdir(exist_ok=True)

test_b = True
roi_color = (255, 0, 0)
bbox_color = (0, 0, 255)

/Users/mnjm/.pyvenv/dl_default/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CoreMLExecutionProvider, AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-02-08 22:09:13.859891 [W:onnxruntime:, coreml_execution_provider.cc:115 GetCapability] CoreMLExecutionProvider::GetCapability, number of partitions supported by CoreML: 15 number of nodes in the graph: 320 number of nodes supported by CoreML: 304


In [3]:
color_idx = 0
total_colors = 30

def generate_unique_colors():
    global color_idx
    hue = int((color_idx * 180 / total_colors) % 180)
    color_idx += 1
    saturation, value = 200, 255
    color = np.uint8([[[hue, saturation, value]]])
    bgr_color = cv2.cvtColor(color, cv2.COLOR_HSV2BGR)[0][0]
    return tuple(map(int, bgr_color))

In [4]:
exit_b = False
for vid_path in vid_path_l:
    
    cap = cv2.VideoCapture(vid_path)
    if not cap.isOpened():
        print(f"Error: Reading {vid_path.name} video failed.")
        continue
    
    vid_w, vid_h, fps = map(lambda x: int(cap.get(x)), (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    print(f"Video WxH:{vid_w}x{vid_h} FPS:{fps}")
    
    # Read ROI
    roi_path = str(vid_path.parent / vid_path.stem) + "_roi.txt"
    roi = np.loadtxt(roi_path, dtype=np.int32)
    assert roi.shape == (4,), f"Invalid ROI {roi}"
    
    out_vid_path = out_vid_dir / vid_path.name
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out_vid = cv2.VideoWriter(out_vid_path, fourcc, fps, (vid_w, vid_h))

    det_l = []
    dist_thresh = 40
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break 
        
        x1, y1, x2, y2 = roi
        img = frame[y1:y2, x1:x2, :]
        bbox_l = model.detect(img)
        cv2.rectangle(frame, roi[:2], roi[2:], roi_color, 2)
        new_det_l = []
        for bbox_idx, bbox in enumerate(bbox_l):
            center = np.array([(bbox.x1 + bbox.x2) / 2, (bbox.y1 + bbox.y2) / 2])
            # calc dist from all known objects
            min_idx, min_dist = -1, np.inf
            for idx, det in enumerate(det_l):
                d_center = det['center']
                dist = np.linalg.norm(center - d_center)
                if dist < min_dist:
                    min_idx, min_dist = idx, dist
            print(min_dist)
            if min_dist < dist_thresh:
                color = det_l[min_idx]['color']
                frame_count = det_l[min_idx]['frame_count'] + 1
            else:
                color = generate_unique_colors()
                frame_count = 1
            new_det_l.append({
                'color': color,
                'frame_count': frame_count,
                'center': center,
                'bbox_idx': bbox_idx,
            })
        
        det_l = new_det_l
        for det in det_l:
            bbox = bbox_l[det['bbox_idx']]
            color = det['color']
            disp_bbox = np.array([bbox.x1, bbox.y1, bbox.x2, bbox.y2], dtype=np.int32).reshape(2, 2) + roi[:2]
            x1, y1, x2, y2 = disp_bbox.reshape(-1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
        out_vid.write(frame)
        cv2.imshow('show', frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('n'):
            break
        if key == ord('q'):
            exit_b = True
            break
    
    cap.release()
    out_vid.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    if exit_b:
        break

Video WxH:3840x2160 FPS:23
inf
inf
inf
inf
inf
inf
inf
75.11739291199814
36.69326929223586
44.91228090652719
35.74405593861205
38.03292410232232
51.071272874625315
65.6222766771825
71.45354125401524
51.2977216698
38.48068316291396
36.65958565419028
49.48132855273111
37.57513905472194
61.373361394260996
70.533203125
36.65958565419029
36.191152016633524
62.6721736546098
33.89485619379001
45.81352854699505
41.24703576968243
73.29073216654879
36.69326929223586
43.05676941145119
35.73325359871732
36.649053168458956
62.75199224371865
25.43077876315814


2025-02-08 22:09:16.907 Python[7336:93218] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-08 22:09:16.907 Python[7336:93218] +[IMKInputSession subclass]: chose IMKInputSession_Modern


70.53429780972508
34.81081187348286
34.82855177578919
36.649053168458956
61.373361394260996
53.128997081148576
26.34440935471573
73.2854644477089
47.63372429704399
36.67432605251595
36.67432605251595
59.542312401308116
26.57798843487816
36.64273222385445
71.45354125401524
48.56085406092823
36.649053168458956
66.18212890625
33.91307523515845
40.23838060309548
589.5671545090545
74.19830625199799
34.817465396068755
36.69326929223586
51.30148447762949
39.135278391766
62.289372394000935
27.066284828436693
71.45354125401524
34.84406678142154
46.718449617881014
36.649053168458956
35.778601509257626
61.373361394260996
25.68021523124008
73.2823036346496
33.90168948169301
461.62676446676164
16.60493991380323
23.816608872642718
35.802332253381124
47.174906982750656
64.24040358753389
73.29073216654879
36.649053168458956
26.62381857459302
18.324526584229478
35.75917373223216
42.138551131417394
65.49629607728085
50.382391922198394
73.2854644477089
16.530373211920853
32.98592687905423
37.556640625
49

In [5]:
cv2.destroyAllWindows()
cv2.waitKey(1)

-1